# Custom ORM

In [14]:
class Base():
    pass

class Column():
    def __init__(self, dtype):
        pass

class relationship():
    def __init__(self, children_model_class):
        pass

Make this work:

In [ ]:
class Email(Base):
    value = Column(str)
    
    def __init__(self, *args, **kwargs):
        pass
    
class User(Base):
    name = Column(str)
    emails = relationship('Email')
    
    def __init__(self, *args, **kwargs):
        pass
    
    def __repr__(self):
        return '<user CP>'
    
user = User(name='CP')
user.emails = [
    Email(value='cp1@od.hk'),
    Email(value='cp2@od.hk')
]

assert str(user) == '<user CP>'

# Class Instantiation

In [1]:
class Foo(object):
    def __init__(self, x, y=0):
        self.x = x
        self.y = y

In [4]:
# foo = Foo(1, y=2)
foo = Foo.__call__(1, y=2)
print(foo)
print(foo.x)
print(foo.y)

1
2


In [28]:
# simulating type's __call__()

class FooMetaclass(type):    
    def __call__(self, *args, **kwargs):
        obj = self.__new__(self, *args, **kwargs)
        if obj is not None:
            obj.__init__(*args, **kwargs)
        return obj

class Foo(metaclass=FooMetaclass):
    def __init__(self, x, y=0):
        self.x = x
        self.y = y

foo = Foo(1, y=2)

print(foo)
print(foo.x)
print(foo.y)

1
2


1. `Foo(*args, **kwargs)` is equivalent to `Foo.__call__(*args, **kwargs)`
2. Since `Foo` is an instance of `type`, `Foo.__call__(*args, **kwargs)` calls `type.__call__(Foo, *args, **kwargs)`
3. `type.__call__(Foo, *args, **kwargs)` calls `type.__new__(Foo, *args, **kwargs)` which returns `obj`
4. `obj` is then initialized by calling `obj.__init__(*args, **kwargs)`
5. `obj` is returned

# Attribute Access

In [7]:
class Foo():
    def __init__(self):
        self.bar = 'baz'
        
foo = Foo()

# foo.bar
print(foo.__getattribute__('bar'))

baz


In [17]:
# simulating __getattribute__()

class Foo():
    def __init__(self):
        self.bar = 'baz'
        
    def __getattr__(self, item):
        return 'qux'
        
    def __my_getattribute__(self, item):
        if item in self.__dict__:
            return self.__dict__[item]
        return self.__getattr__(item)

foo = Foo()

# foo.bar
print(foo.__my_getattribute__('bar'))

baz


# Metaclass

In [26]:
class Foo(object):
    bar = 'baz'
print(Foo.bar)

baz


is equivalent to

In [25]:
# type(name, bases, attrs)
Foo = type('Foo', (object,), { 'bar': 'baz' })
print(Foo.bar)

baz


## Create a class automatically transform props to uppercase

# Column Descriptor

In [28]:
class Column():
    def __init__(self, dtype):
        self.dtype = dtype
        self.data = None
    
    def __get__(self, obj, type):
        return self.dtype(self.data)
        
    def __set__(self, obj, value):
        self.data = value

class User():
    name = Column(str)
    
    def __repr__(self):
        return f'<user {self.name}>'

user = User()
user.name = 'CP2'
print(user)

<user CP2>


# Column Descriptor + Base Metaclass

In [43]:
class BaseMeta(type):
#     def __new__(mcls, name, base, attribs):
#         print(attribs.get('__annotations__',{}))
#         return super().__new__(mcls, name, base, attribs)
    def __call__(self, *args, **kwargs):
        print('BaseMeta.__call__')
        instance = self.__new__(self, *args, **kwargs)
        self.__init__(instance, *args, **kwargs)
        for column_name, column_value in kwargs.items():
            setattr(self, column_name, column_value)
        return instance


# class Base(metaclass=BaseMeta):
class Base():
    def __init__(self, **kwargs):
        print('Base.__init__')
        for column_name, column_value in kwargs.items():
            setattr(self, column_name, column_value)


class Column():
    def __init__(self, dtype):
        self.dtype = dtype
        self.data = None
    
    def __get__(self, obj, type_):
        return self.dtype(self.data)
        
    def __set__(self, obj, value):
        self.data = value

class User(Base):
    name = Column(str)
    
    def __repr__(self):
        return f'<user {self.name}>'

user = User(name='CP')
# user.name = 'CP2'
print(user)

Base.__init__
<user CP>
